# model save / load

### 생성한 모델을 disk 에 저장하고 복원

- model 의 architecture 저장 : json, yaml format  

- model 의 weigth 저장 : h5 file format  

- hdf5(Hierarchical Data Format version 5)는 대용량 데이터를 저장하기 위한 파일 포맷

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

X_train = X_train[:, :, :, np.newaxis]
X_test = X_test[:, :, :, np.newaxis]
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)

# LeNet
model = tf.keras.Sequential()

model.add(Conv2D(6, kernel_size=5, padding="same", input_shape=(28, 28, 1)))
model.add(Activation("relu"))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding="valid"))

model.add(Conv2D(16, kernel_size=5, padding="valid"))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

model.add(Flatten())

model.add(Dense(120))
model.add(Activation("relu"))

model.add(Dense(84))
model.add(Activation("relu"))

model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, epochs=1, validation_data=(X_test, y_test))

score = model.evaluate(X_test, y_test)
print(score[0])
print(score[1])

Instructions for updating:
Colocations handled automatically by placer.
Train on 60000 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
10000/10000 [==============================] - 1s 75us/sample - loss: 0.0654 - acc: 0.9798
0.06536953851068393
0.9798


### Model Architecture Save

In [2]:
json_string = model.to_json()
yaml_string = model.to_yaml()

In [3]:
with open("lenet.json", "w") as json_file:
    json_file.write(json_string)

### weight 만 save

In [4]:
model.save_weights("lenet_weight.h5")

### 전체 model save

In [5]:
model.save("lenet_model.h5")

## Model Load

- architecture load + weight load + compile

In [6]:
from tensorflow.keras.models import model_from_json

json_file = open("lenet.json", "r")
read_model = json_file.read()
json_file.close()

loaded_model = model_from_json(read_model)
loaded_model.load_weights("lenet_weight.h5")

loaded_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [7]:
score = loaded_model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 92us/sample - loss: 0.0654 - acc: 0.9798


In [8]:
from tensorflow.keras.models import load_model

loaded_model = load_model('lenet_model.h5')

In [9]:
score = loaded_model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 84us/sample - loss: 0.0654 - acc: 0.9798


In [10]:
y_predict = model.predict(X_test)
y_predict

array([[3.1072620e-06, 5.0682735e-07, 8.9254841e-05, ..., 9.9979192e-01,
        1.9974200e-07, 2.6895765e-05],
       [9.4027564e-06, 9.4089955e-05, 9.9982280e-01, ..., 3.1291747e-09,
        2.6771559e-05, 8.7529595e-11],
       [1.8905029e-04, 9.9608934e-01, 9.0959930e-04, ..., 4.1674782e-04,
        1.0186188e-03, 1.3086006e-04],
       ...,
       [3.1842017e-08, 1.1226711e-06, 7.0094977e-08, ..., 2.0878047e-06,
        2.1368935e-06, 1.3689404e-04],
       [6.0698005e-07, 5.6845039e-08, 4.9737313e-08, ..., 1.3568047e-08,
        6.8907486e-03, 5.3856746e-07],
       [3.3959093e-06, 8.5748075e-10, 2.7060403e-06, ..., 2.0614845e-11,
        1.2906453e-06, 1.0645661e-09]], dtype=float32)

In [11]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [12]:
np.argmax(y_predict, axis=1)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [13]:
np.argmax(y_test, axis=1)

array([7, 2, 1, ..., 4, 5, 6], dtype=int64)

In [14]:
np.sum(np.argmax(y_predict, axis=1) == np.argmax(y_test, axis=1)) / len(y_test) * 100

97.98

In [15]:
loaded_model.predict_classes(X_test[:1,:])

array([7], dtype=int64)

## 실습 : Model 을 yaml file 로 저장하고 복원 후 weight load 및 compile 및 predict